In [ ]:
    ###########################################################################################################################
#Testing the clasicall detectors , fully_con and detnet under the same data for BPSK modulation with 10x12 fixed channel
###########################################################################################################################

In [7]:
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_eager_execution()

# Data generation and other helper functions (generate_data, zero_forcing_detector, mmse_detector, conjugate_gradient_correct) remain the same

# DetNet Model definition (remains unchanged)
def DetNetModel(K, L, v_size, hl_size, res_alpha, LOG_LOSS, startingLearningRate, decay_step_size, decay_factor):
    """Define and train the DetNet model."""
    HY = tf.compat.v1.placeholder(tf.float32, shape=[None, K], name='HY')
    X = tf.compat.v1.placeholder(tf.float32, shape=[None, K], name='X')
    HH = tf.compat.v1.placeholder(tf.float32, shape=[None, K, K], name='HH')
    X_IND = tf.compat.v1.placeholder(tf.float32, shape=[None, K, 2], name='X_IND')
    batch_size = tf.shape(HY)[0]
    S2=[]
    S2.append(tf.zeros([batch_size,2*K]))
    S1=[]
    S1.append(tf.zeros([batch_size,K]))
    S_NO_res=[]
    S_NO_res.append(tf.zeros([batch_size,K]))
    V=[]
    V.append(tf.zeros([batch_size,v_size]))
    LOSS=[]
    LOSS.append(tf.zeros([]))
    BER=[] 
    BER.append(tf.zeros([]))
    delta = tf.Variable(tf.zeros(L * 2), name='delta')  # Fixed shape
    Z1 = []
    ZZ1 = []
    W11 = []
    w11 = []
    W22 = []
    w22 = [] 
    W33 = []
    w33 = []

    for i in range(1, L):
        temp1 = tf.matmul(tf.expand_dims(S1[-1], 1), HH)
        temp1 = tf.squeeze(temp1, 1)

        Z1 = S1[-1] - delta[(i - 1) * 2] * HY + delta[(i - 1) * 2 + 1] * temp1
        Z = tf.concat([Z1, V[-1]], 1)

        ZZ, Wtemp, wtemp = relu_layer(Z, K + v_size, hl_size, 'relu' + str(i))
        W11.append(Wtemp)
        w11.append(wtemp)
        ZZ1.append((ZZ))

        S2_temp,W22_temp,w22_temp = sign_layer(ZZ, hl_size, 2*K, 'sign' + str(i))
        S2.append(S2_temp)
        W22.append(W22_temp)
        w22.append(w22_temp)
        

        S_NO_res.append(S2[i])
        S2[i]=(1-res_alpha)*S2[i]+res_alpha*S2[i-1]
        V_temp ,  W33_temp , w33_temp=affine_layer(ZZ , hl_size , v_size,'aff'+str(i))
        V.append(V_temp)
        W33.append(W33_temp)
        w33.append(w33_temp)

        V[i]=(1-res_alpha)*V[i]+res_alpha*V[i-1]  
    
        S3 = tf.reshape(S2[i],[batch_size,K,2])
    
        temp_0 = S3[:,:,0]
        temp_1 = S3[:,:,1]
    
        temp_2 = 1*temp_0 + (-1)*temp_1
        S1.append(temp_2)

        X_IND_reshaped = tf.reshape(X_IND, [batch_size, 2 * K])
        if LOG_LOSS == 1:
            LOSS.append(tf.reduce_mean(tf.square(X_IND_reshaped - S2[-1])) * tf.cast(i, tf.float32))
        else:
            LOSS.append(tf.reduce_mean(tf.square(X_IND_reshaped - S2[-1])))

        BER.append(tf.reduce_mean(tf.cast(tf.not_equal(X_IND, tf.round(S3)), tf.float32)))

    TOTAL_LOSS = tf.add_n(LOSS)

    saver = tf.compat.v1.train.Saver()
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(startingLearningRate, global_step, decay_step_size, decay_factor, staircase=True)
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(TOTAL_LOSS)

    init_op = tf.compat.v1.global_variables_initializer()
    return saver, train_step, init_op, HY, HH, X, X_IND, batch_size, S3, TOTAL_LOSS, BER

# Neural Network Fully Connected model definition
def build_fc_nn(NNinput, batchSize, N, K, fc_size):
    W_fc1 = tf.Variable(tf.truncated_normal([N, fc_size], stddev=0.05))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[fc_size]))
    h_fc1 = tf.nn.relu(tf.matmul(NNinput, W_fc1) + b_fc1)

    W_fc2 = tf.Variable(tf.truncated_normal([fc_size, fc_size], stddev=0.05))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[fc_size]))
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

    W_fc3 = tf.Variable(tf.truncated_normal([fc_size, fc_size], stddev=0.05))
    b_fc3 = tf.Variable(tf.constant(0.1, shape=[fc_size]))
    h_fc3 = tf.nn.relu(tf.matmul(h_fc2, W_fc3) + b_fc3)

    W_fc4 = tf.Variable(tf.truncated_normal([fc_size, fc_size], stddev=0.05))
    b_fc4 = tf.Variable(tf.constant(0.1, shape=[fc_size]))
    h_fc4 = tf.nn.relu(tf.matmul(h_fc3, W_fc4) + b_fc4)

    W_fc5 = tf.Variable(tf.truncated_normal([fc_size, fc_size], stddev=0.05))
    b_fc5 = tf.Variable(tf.constant(0.1, shape=[fc_size]))
    h_fc5 = tf.nn.relu(tf.matmul(h_fc4, W_fc5) + b_fc5)

    # Output layer
    W_fc6 = tf.Variable(tf.truncated_normal([fc_size, 2*K], stddev=0.05))
    b_fc6 =tf.Variable(tf.constant(0.1, shape=[2 * K]))
    h_fc6 = tf.matmul(h_fc5, W_fc6) + b_fc6

    output = tf.reshape(h_fc6, [batchSize, K, 2])

    return output
def test_detnet_and_fc_nn(snrdb_low_test, snrdb_high_test, H, N, K, B, test_iter, train_iter, num_snr):
    fc_size = 300
    L = 30 # Increase number of layers in DetNet
    v_size = 4 * K  # Dimension of the auxiliary variable in DetNet
    hl_size = 4 * K  # Hidden layer size for DetNet
    res_alpha = 0.9 # Adjust residual scaling
    LOG_LOSS = 1
    startingLearningRate = 0.0005  # Reduce the learning rate
    decay_step_size = 1000
    decay_factor = 0.95  # More gradual learning rate decay
    detnet_train_batch_size = B

    # Build DetNet model
    saver_detnet, train_step_detnet, init_op_detnet, HY_detnet, HH_detnet, X_detnet, X_IND_detnet, batch_size_detnet, S3_detnet,loss_detnet, ber_detnet = DetNetModel(K, L, v_size, hl_size, res_alpha, LOG_LOSS, startingLearningRate, decay_step_size, decay_factor)

    # Build FC-NN model
    NNinput = tf.compat.v1.placeholder(tf.float32, shape=[None, N])
    org_signal = tf.compat.v1.placeholder(tf.float32, shape=[None, K])
    X_IND = tf.compat.v1.placeholder(tf.float32, shape=[None, K, 2])
    batchSize = tf.compat.v1.placeholder(tf.int32)

    output_fc_nn = build_fc_nn(NNinput, batchSize, N, K, fc_size)
    ssd_fc_nn = tf.reduce_sum(tf.reduce_sum(tf.square(X_IND - output_fc_nn)))
    temp_0 = output_fc_nn[:, :, 0]
    temp_1 = output_fc_nn[:, :, 1]
    output2_fc_nn = 1 * temp_0 + (-1) * temp_1
    rounded_fc_nn = tf.sign(output2_fc_nn)
    eq_fc_nn = tf.equal(rounded_fc_nn, org_signal)
    eq2_fc_nn = tf.reduce_sum(tf.cast(eq_fc_nn, tf.int32))
    train_step_fc_nn = tf.train.AdamOptimizer(0.0003).minimize(ssd_fc_nn)

    # Use a single session for both models
    with tf.Session() as sess:
        # Initialize variables
        sess.run(init_op_detnet)
        sess.run(tf.global_variables_initializer())

        # Training DetNet model
        for i in range(train_iter):
            batch_Y, batch_H, batch_HY, batch_HH, batch_X, SNR1, x_ind,tmp_snr = generate_data(B, K, N, snrdb_low_test, snrdb_high_test , H)
            sess.run(train_step_detnet, feed_dict={HY_detnet: batch_HY, HH_detnet: batch_HH, X_detnet: batch_X, X_IND_detnet: x_ind})
            if i % 500== 0:
                loss_val = sess.run(loss_detnet, feed_dict={HY_detnet: batch_HY, HH_detnet: batch_HH, X_detnet: batch_X, X_IND_detnet: x_ind})
                print(f'DetNet Training Step {i}, Loss: {loss_val}')

        # Training FC-NN model
        for i in range(train_iter):
            batch_Y, batch_H, batch_HY, batch_HH, batch_X, SNR1, x_ind,tmp_snr = generate_data(B, K, N, snrdb_low_test , snrdb_high_test , H)
            sess.run(train_step_fc_nn, feed_dict={NNinput: batch_Y, org_signal: batch_X, batchSize: B, X_IND: x_ind})
            if i % 500== 0:
                eq2_val = sess.run(eq2_fc_nn, feed_dict={NNinput: batch_Y, org_signal: batch_X, batchSize: B, X_IND: x_ind})
                batch_ber = (K * B - eq2_val) / (K * B)
                print(f'FC-NN Training Step {i}, BER: {batch_ber}')

        # Testing phase
        bers_detnet = np.zeros((1, num_snr))
        bers_fc_nn = np.zeros((1, num_snr))
        bers_zf = np.zeros((1, num_snr))
        bers_mmse = np.zeros((1, num_snr))
        bers_cg = np.zeros((1, num_snr))

        snrdb_list = np.linspace(1.0, 20.0, num_snr)
        snr_list = 10.0 ** (snrdb_list / 10.0)

        for i_snr in range(num_snr):
            Cur_SNR=snr_list[i_snr]
            print(f"Testing at SNR {Cur_SNR}")
            for i in range(test_iter):
                batch_Y, batch_H, batch_HY, batch_HH, batch_X, SNR1, x_ind,tmp_snr= generate_data(1000, K, N, Cur_SNR, Cur_SNR, H)

                # DetNet Testing
                S3_val = sess.run(S3_detnet, feed_dict={HY_detnet: batch_HY, HH_detnet: batch_HH, X_detnet: batch_X, X_IND_detnet: x_ind})
                incorrect_bits_detnet = np.sum(np.not_equal(np.sign(S3_val), x_ind))
                batch_ber_detnet = incorrect_bits_detnet / (K * 1000)
                bers_detnet[0][i_snr] += batch_ber_detnet / test_iter

                # FC-NN Testing
                eq2_val = sess.run(eq2_fc_nn, feed_dict={NNinput: batch_Y, org_signal: batch_X, batchSize: 1000, X_IND: x_ind})
                incorrect_bits_fc_nn = K * 1000 - eq2_val
                batch_ber_fc_nn = incorrect_bits_fc_nn / (K * 1000)
                bers_fc_nn[0][i_snr] += batch_ber_fc_nn / test_iter

                # Zero Forcing Detection
                x_hat_zf = np.zeros((1000, K))
                for b in range(1000):
                    x_hat_zf[b, :] = zero_forcing_detector(batch_H[b, :, :], batch_Y[b, :])
                rounded_zf = np.sign(x_hat_zf)
                incorrect_bits_zf = np.sum(np.not_equal(rounded_zf, batch_X))
                batch_ber_zf=incorrect_bits_zf/(K * 1000)
                bers_zf[0][i_snr] += batch_ber_zf /test_iter

                # MMSE Detection
                x_hat_mmse = np.zeros((1000, K))
                for b in range(1000):
                    x_hat_mmse[b, :] = mmse_detector(batch_H[b, :, :], batch_Y[b, :], tmp_snr/ Cur_SNR)
                rounded_mmse = np.sign(x_hat_mmse)
                incorrect_bits_mmse = np.sum(np.not_equal(rounded_mmse, batch_X))
                batch_ber_mmse=incorrect_bits_mmse/(K*1000)
                bers_mmse[0][i_snr] += batch_ber_mmse/ test_iter
                
                # Conjugate Gradient Detection
                x_hat_cg = np.zeros((1000, K))
                s_init = np.zeros(K)
                for b in range(1000):
                    x_hat_cg[b, :] = conjugate_gradient_correct(batch_HH[b, :, :], batch_HY[b, :], s_init, max_iterations=1000, tolerance=1e-6)[0][:, -1]
                rounded_cg = np.sign(x_hat_cg)
                incorrect_bits_cg = np.sum(np.not_equal(rounded_cg, batch_X))
                bers_cg[0][i_snr] += incorrect_bits_cg / (K * 1000* test_iter)

        print(f"BER Results for DetNet: {bers_detnet}")
        print(f"BER Results for FC-NN: {bers_fc_nn}")
        print(f"BER Results for Zero Forcing: {bers_zf}")
        print(f"BER Results for MMSE: {bers_mmse}")
        print(f"BER Results for Conjugate Gradient: {bers_cg}")

# Test both models and detectors
snr_low_= 1.0
snr_high_= 20.0
snr_low=20**(snr_low_/10.0)
snr_high=20**(snr_high_/10.0)
N = 12
K = 12
B =1000
test_iter = 100
train_iter = 10000
num_snr = 10
H = np.random.randn(N, K) 


test_detnet_and_fc_nn(snr_low, snr_high, H, N, K, B, test_iter, train_iter, num_snr)


DetNet Training Step 0, Loss: 217.48635864257812
DetNet Training Step 500, Loss: 108.69695281982422
DetNet Training Step 1000, Loss: 76.4346923828125
DetNet Training Step 1500, Loss: 58.873138427734375
DetNet Training Step 2000, Loss: 48.48405456542969
DetNet Training Step 2500, Loss: 39.982452392578125
DetNet Training Step 3000, Loss: 34.21874237060547
DetNet Training Step 3500, Loss: 28.26943588256836
DetNet Training Step 4000, Loss: 23.192642211914062
DetNet Training Step 4500, Loss: 20.69095802307129
DetNet Training Step 5000, Loss: 19.244014739990234
DetNet Training Step 5500, Loss: 17.37433624267578
DetNet Training Step 6000, Loss: 15.781689643859863
DetNet Training Step 6500, Loss: 14.876791000366211
DetNet Training Step 7000, Loss: 14.21640396118164
DetNet Training Step 7500, Loss: 13.70832633972168
DetNet Training Step 8000, Loss: 12.419853210449219
DetNet Training Step 8500, Loss: 11.797231674194336
DetNet Training Step 9000, Loss: 11.271268844604492
DetNet Training Step 9500

In [ ]:
####################################################################################
#help functions for the main code 
###################################################################################

In [2]:
def generate_data(B,K,N,snr_low,snr_high,H_org):
    x_=np.sign(np.random.rand(B,K)-0.5)
    y_=np.zeros([B,N])
    w=np.random.randn(B,N)
    Hy_=x_*0
    H_ = np.zeros([B,N,K])
    HH_= np.zeros([B,K,K])
    SNR_= np.zeros([B])
    x_ind = np.zeros([B,K,2])
    for i in range(B):
        for ii in range(K):
            if x_[i][ii] == 1:
                x_ind[i][ii][0] = 1
            if x_[i][ii] == -1:
                x_ind[i][ii][1] = 1  
    for i in range(B):
        #print i
        SNR = np.random.uniform(low=snr_low,high=snr_high)
        H=H_org
        tmp_snr=(H.T.dot(H)).trace()/K
        #H=H/np.sqrt(tmp_snr)*np.sqrt(SNR)
        H_[i,:,:]=H
        y_[i,:]=(H.dot(x_[i,:])+w[i,:]*np.sqrt(tmp_snr)/np.sqrt(SNR))
        Hy_[i,:]=H.T.dot(y_[i,:])
        HH_[i,:,:]=H.T.dot( H_[i,:,:])
        SNR_[i] = SNR
    return y_,H_,Hy_,HH_,x_,SNR_,x_ind,tmp_snr

# Disable eager execution for compatibility with TensorFlow v1.x
def piecewise_linear_soft_sign(x):
    """Custom soft sign activation."""
    t = tf.Variable(0.1, trainable=False)
    y = -1 + tf.nn.relu(x + t) / (tf.abs(t) + 0.00001) - tf.nn.relu(x - t) / (tf.abs(t) + 0.00001)
    return y

def affine_layer(x, input_size, output_size, layer_num):
    """Affine layer for the neural network."""
    W = tf.Variable(tf.random_normal([input_size, output_size], stddev=0.01), name='W' + layer_num)
    w = tf.Variable(tf.random_normal([1, output_size], stddev=0.01), name='w' + layer_num)
    y = tf.matmul(x, W) + w
    return y, W, w

def relu_layer(x, input_size, output_size, layer_num):
    """ReLU layer."""
    y, W, w = affine_layer(x, input_size, output_size, layer_num)
    y = tf.nn.relu(y)
    return y, W, w

def sign_layer(x, input_size, output_size, layer_num):
    """Sign activation layer."""
    y, W, w = affine_layer(x, input_size, output_size, layer_num)
    return y, W, w
# Zero Forcing Detector remains unchanged
def zero_forcing_detector(H, y):
    H_T_H = np.dot(H.T, H)
    H_T_H_inv = np.linalg.inv(H_T_H)
    H_T_y = np.dot(H.T, y)
    x_hat = np.dot(H_T_H_inv, H_T_y)
    return x_hat

# MMSE Detector remains unchanged
def mmse_detector(H, y, sigma2):
    Nt = H.shape[1]
    H_T_H = np.dot(H.T, H)
    I = np.eye(Nt)
    H_T_H_inv = np.linalg.inv(H_T_H + sigma2 * I)
    H_T_y = np.dot(H.T, y)
    x_hat_mmse = np.dot(H_T_H_inv, H_T_y)
    return x_hat_mmse

# Conjugate Gradient Correct function remains unchanged
def conjugate_gradient_correct(A, B, s, max_iterations, tolerance,sigma_square=0):
    A=A+sigma_square
    n = len(B)
    s_exp = np.zeros((n, max_iterations + 1))
    s_exp[:, 0] = s
    rk = B - A @ s
    dk = rk
    iter_max = 0

    for iter in range(max_iterations):
        alpha = np.dot(rk, rk) / np.dot(dk, A @ dk)
        s = s + alpha * dk
        rk_new = rk - alpha * A @ dk
        iter_max += 1

        if np.linalg.norm(rk_new) < tolerance:
            s_exp[:, iter + 1] = s
            s_exp = s_exp[:, :iter + 2]
            break

        beta = np.dot(rk_new, rk_new) / np.dot(rk, rk)
        dk = rk_new + beta * dk
        rk = rk_new
        s_exp[:, iter + 1] = s

    if iter == max_iterations - 1:
        s_exp = s_exp[:, :max_iterations + 1]

    return s_exp, iter_max
